In [1]:
import os

os.chdir("..")
print(f"Changed working directory to: {os.getcwd()}")

Changed working directory to: /home/jovyan/work/FlareSense


In [2]:
import os

os.getcwd()

'/home/jovyan/work/FlareSense'

In [3]:
import torch
import mlflow
import dagshub
import torchmetrics
import src.utils.data as data
import pytorch_lightning as pl

from torchvision import transforms
from tqdm.notebook import tqdm
from src.models.EfficientNetV2SBinaryClassifier import EfficientNetV2SBinaryClassifier

mlflow.pytorch.autolog()
torch.set_float32_matmul_precision("high")

/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:128: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
2023/11/13 08:44:22 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version, or try upgrading MLflow.
2023-11-13 08:44:22.478418: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI 

## Training

Defining the data and model

In [4]:
model = EfficientNetV2SBinaryClassifier(lr=1e-3, weight_decay=1e-4)
data_folder_path = "data/raw/burst_images/"

data_module = data.ECallistoDataModule(
    data_folder=data_folder_path,
    transform=transforms.Compose(
        [
            transforms.ToPILImage(),
            transforms.Resize((224, 224), antialias=True), # ViT setzt 224 voraus, bisher 193x240
            transforms.ToTensor(),
        ]
    ),
    batch_size=64,
    num_workers=10,
    val_ratio=0.15,
    test_ratio=0.15,
    split_by_date=True,
    filter_instruments=["australia_assa_02"],
)
data_module.setup()

Training and logging the model

In [5]:
dagshub.init("FlareSense", "FlareSense", mlflow=True)
mlflow.start_run()

run_id = mlflow.active_run().info.run_id
print(f"Run ID: {run_id}")
print(f"Link: https://dagshub.com/FlareSense/FlareSense/experiments/#/experiment/m_{run_id}")

trainer = pl.Trainer(max_epochs=50, log_every_n_steps=1)

trainer.fit(
    model,
    train_dataloaders=data_module.train_dataloader(),
    val_dataloaders=data_module.val_dataloader(),
)

trainer.test(model, dataloaders=data_module.test_dataloader())

mlflow.end_run()

Repository initialized!

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Run ID: 623f6e8416374806affb54d9b47319ed
Link: https://dagshub.com/FlareSense/FlareSense/experiments/#/experiment/m_623f6e8416374806affb54d9b47319ed


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type            | Params
------------------------------------------------------
0 | efficientnet_v2_s | EfficientNet    | 20.2 M
1 | precision         | BinaryPrecision | 0     
2 | recall            | BinaryRecall    | 0     
------------------------------------------------------
18.4 M    Trainable params
1.8 M     Non-trainable params
20.2 M    Total params
80.715    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.
2023/11/13 09:16:56 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/conda/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.01164815854281187    │
│      test_precision       │     0.795918345451355     │
│        test_recall        │    0.9512194991111755     │
└───────────────────────────┴───────────────────────────┘

## Testing

Downloading the model from mlflow and putting it on the correct device

In [6]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

mlflow.set_tracking_uri('https://dagshub.com/FlareSense/Flaresense.mlflow')
model = mlflow.pytorch.load_model(f"runs:/{run_id}/model/").to(device)
model.eval()

EfficientNetV2SBinaryClassifier(
  (efficientnet_v2_s): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): FusedMBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
          )
          (stochastic_depth): StochasticDepth(p=0.0, mode=row)
        )
        (1): FusedMBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 

Run through testdata

In [7]:
test_labels_list = []
test_preds_list = []
with torch.no_grad():
    for batch in tqdm(data_module.test_dataloader()):
        images, info = batch
        images = images.to(device)
        binary_labels = [0 if label == "no_burst" else 1 for label in info['label']]
        binary_labels = torch.tensor(binary_labels).int().view(-1, 1)
        binary_labels = binary_labels.to(device)
        
        outputs = model(images.expand(-1, 3, -1, -1))
        predictions = (outputs >= 0.5).int()
        
        test_labels_list.append(binary_labels)
        test_preds_list.append(predictions)

  0%|          | 0/52 [00:00<?, ?it/s]

Create confmatrix

In [8]:
test_labels = torch.cat(test_labels_list, dim=0)
test_preds = torch.cat(test_preds_list, dim=0)

confmat_metric = torchmetrics.ConfusionMatrix(num_classes=2, task="binary").to(device)
confmat_metric(test_preds, test_labels)

tensor([[3230,   10],
        [   2,   39]], device='cuda:0')